In [10]:
import common

import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

from dotenv import load_dotenv
from datetime import datetime
from zoneinfo import ZoneInfo
import wandb

In [11]:
is_soft_voting = True
augment_ratio = 1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [12]:

# ljm 이 생성한 모델 checkpoint
cp_path_list = [
   'model_bak7/cp-tiny_vit_21m_512.dist_in22k_ft_in1k_sd_3751_epc_5_aug_200_vl_0.1719_va_0.9573_vf1_0.9558_fold_1_folds_2_PICK.pt',
   'model_bak7/cp-tiny_vit_21m_512.dist_in22k_ft_in1k_sd_3751_epc_9_aug_200_vl_0.2360_va_0.9458_vf1_0.9460_fold_2_folds_2_PICK.pt',
]

In [13]:
# jbc 님이 생성한 모델 (checkpoint 아님) 섭미 점수 0.9531
not_cp_info_list = [
    {
        'path': 'model_jbc/efficientnet_b3.ra2_in1k_256SIZE_16BATCH_5EPOCH_200AUG_best_model.pth',
        'arch': 'efficientnet_b3.ra2_in1k',
        'img_size': 256,
        'batch_size': 16,
    },
    {
        'path': 'model_jbc/efficientnet_b5.sw_in12k_ft_in1k_384SIZE_16BATCH_5EPOCH_200_AUG_0.8_best_model.pth',
        'arch': 'efficientnet_b5.sw_in12k_ft_in1k',
        'img_size': 384,
        'batch_size': 16,
    },
    {
        'path': 'model_jbc/efficientnetv2_rw_m.agc_in1k_256SIZE_16BATCH_5EPOCH_200AUG_best_model.pth',
        'arch': 'efficientnetv2_rw_m.agc_in1k',
        'img_size': 256,
        'batch_size': 16,
    },
    {
        'path': 'model_jbc/tiny_vit_21m_384.dist_in22k_ft_in1k_384SIZE_16BATCH_5EPOCH_200_AUG_0.9_best_model.pth',
        'arch': 'tiny_vit_21m_384.dist_in22k_ft_in1k',
        'img_size': 384,
        'batch_size': 16,
    }      
]

In [14]:
model_list = []
tst_loader_list = []

In [15]:

for cp_path in cp_path_list:
    checkpoint = torch.load(cp_path, map_location = device)
    
    print(checkpoint['model'], checkpoint['tst_img_size'], checkpoint['batch_size'])
    
    model = timm.create_model(
        checkpoint['model'],
        pretrained = False,
        num_classes = 17,
    ).to(device)
    
    model.load_state_dict(checkpoint['model_state_dict'])    
    model_list.append(model)
    
    ##
    tst_transform = common.create_tst_transform(checkpoint['tst_img_size'])
    tst_aug_transform = common.create_tst_aug_transform(checkpoint['tst_img_size'])

    tst_dataset = common.ImageDataset(
        "datasets_fin/sample_submission.csv",
        "datasets_fin/test/",
        transform = tst_transform, 
        aug_transform = tst_aug_transform, 
        augment_ratio = augment_ratio
    )

    tst_loader = DataLoader(
        tst_dataset,
        batch_size = checkpoint['batch_size'],
        shuffle = False,
        num_workers = 2,
        pin_memory = True
    )
    
    tst_loader_list.append(tst_loader)

print(len(model_list), len(tst_loader_list))

tiny_vit_21m_512.dist_in22k_ft_in1k 512 24
tiny_vit_21m_512.dist_in22k_ft_in1k 512 24
2 2


In [16]:

for not_cp_info in not_cp_info_list:
    print(not_cp_info['arch'], not_cp_info['img_size'], not_cp_info['batch_size'])
    
    model = timm.create_model(
        not_cp_info['arch'],
        pretrained = False,
        num_classes = 17,
    ).to(device)
    
    model.load_state_dict(torch.load(not_cp_info['path'], map_location = device))
    model_list.append(model)
    
    ##
    tst_transform = common.create_tst_transform(not_cp_info['img_size'])
    tst_aug_transform = common.create_tst_aug_transform(not_cp_info['img_size'])

    tst_dataset = common.ImageDataset(
        "datasets_fin/sample_submission.csv",
        "datasets_fin/test/",
        transform = tst_transform,
        aug_transform = tst_aug_transform, 
        augment_ratio = augment_ratio
    )

    tst_loader = DataLoader(
        tst_dataset,
        batch_size = not_cp_info['batch_size'],
        shuffle = False,
        num_workers = 2,
        pin_memory = True
    )
    
    tst_loader_list.append(tst_loader)

print(len(model_list), len(tst_loader_list))

efficientnet_b3.ra2_in1k 256 16
efficientnet_b5.sw_in12k_ft_in1k 384 16
efficientnetv2_rw_m.agc_in1k 256 16
tiny_vit_21m_384.dist_in22k_ft_in1k 384 16
6 6


In [17]:
preds_list = []

for model, tst_loader in zip(model_list, tst_loader_list):
    preds = common.get_preds_list_by_tst_loader(model, tst_loader, device, is_soft_voting)
    preds_list.append(preds)

100%|██████████| 197/197 [00:11<00:00, 17.65it/s]


In [18]:
def get_all_targets_count():
    sample_submission_df = pd.read_csv("datasets_fin/sample_submission.csv")
    return len(sample_submission_df)
    
def hard_voting(predictions):
    predictions = np.asarray(predictions)
    return np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=predictions)

def soft_voting(predictions):
    predictions = np.asarray(predictions)
    mean_axis0 = np.mean(predictions, axis=0)
    
    # 증강된 데이터에 대한 예측값도 고려하기.
    all_targets_count = get_all_targets_count()
    
    aug_size = len(mean_axis0) / all_targets_count
    assert len(mean_axis0) % all_targets_count == 0
    aug_size = int(aug_size)
    
    if aug_size > 1:
        bulk_list = []
        step = 0
        
        for i in range(0, aug_size):
            bulk_list.append(mean_axis0[step:step + all_targets_count])
            step += all_targets_count
        
        bulk_list = np.asarray(bulk_list)
        mean_axis0 = np.mean(bulk_list, axis=0)
        
    return mean_axis0.argmax(axis=1)

# 최종 예측
if is_soft_voting:
    final_pred = soft_voting(preds_list)
else:
    final_pred = hard_voting(preds_list)

# # csv 로 저장
csv_filename = 'pred_ensemble.csv'
common.preds_list_to_save_to_csv(final_pred, tst_loader, csv_filename)
print(f'Prediction save to {csv_filename}')

Prediction save to pred_ensemble.csv
